In [1]:
import builtins
import copy

from selenium import webdriver
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from itertools import combinations
from datetime import datetime

import time

In [2]:
def print(*args, **kwargs):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    builtins.print(f"[{timestamp}] ", *args, **kwargs)

In [3]:
EDGE_DRIVER_PATH = "edgedriver_win32/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [4]:
wait = WebDriverWait(driver, 10)

In [5]:
driver.get("https://rustypot.com/coinflip")

In [6]:
def get_possible_bet_values(inventory_items_values, max_items=5):
    print(f"there are {len(inventory_items_values)} items in inventory")
    possible_bet_values = {}
    for r in range(1, max_items):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = list(combo)
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))
    
    return possible_bet_values_sorted

In [7]:
def get_inventory_items():
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "createCoinflipButton"))
    )
    create_a_game_buttom.click()
    
    try:
        inventory_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "InventoryItem"))
        )
        
        # inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
        inventory_items_values = []
        for inventory_item in inventory_items:
            inventory_items_values.append(float(inventory_item.text[1:]))
        
        driver.get("https://rustypot.com/coinflip")

        print(f"inventory value: {sum(inventory_items_values):.2f}\ninventory items gathered: {inventory_items_values}", )
        
        return inventory_items_values
    except TimeoutException:
        print(f"Timed out waiting for inventory items")
        driver.get("https://rustypot.com/coinflip")
        get_inventory_items()

In [8]:
inventory_items_values = [62.31, 31.75, 31.75, 20.86, 17.38, 13.34, 13.34]

# inventory_items = [i for i in range(1, 21, 3)]
possible_bet_values = get_possible_bet_values(inventory_items_values)
possible_bet_values

[2025-03-19 12:48:31]  there are 7 items in inventory


{13.34: [13.34],
 17.38: [17.38],
 20.86: [20.86],
 26.68: [13.34, 13.34],
 30.72: [17.38, 13.34],
 31.75: [31.75],
 34.2: [20.86, 13.34],
 38.239999999999995: [20.86, 17.38],
 44.06: [17.38, 13.34, 13.34],
 45.09: [31.75, 13.34],
 47.540000000000006: [20.86, 13.34, 13.34],
 49.129999999999995: [31.75, 17.38],
 51.58: [20.86, 17.38, 13.34],
 52.61: [31.75, 20.86],
 58.43000000000001: [31.75, 13.34, 13.34],
 62.31: [62.31],
 62.47: [31.75, 17.38, 13.34],
 63.5: [31.75, 31.75],
 64.92: [20.86, 17.38, 13.34, 13.34],
 65.95: [31.75, 20.86, 13.34],
 69.99: [31.75, 20.86, 17.38],
 75.65: [62.31, 13.34],
 75.81: [31.75, 17.38, 13.34, 13.34],
 76.84: [31.75, 31.75, 13.34],
 79.29: [31.75, 20.86, 13.34, 13.34],
 79.69: [62.31, 17.38],
 80.88: [31.75, 31.75, 17.38],
 83.17: [62.31, 20.86],
 83.33: [31.75, 20.86, 17.38, 13.34],
 84.36: [31.75, 31.75, 20.86],
 88.99000000000001: [62.31, 13.34, 13.34],
 90.18: [31.75, 31.75, 13.34, 13.34],
 93.03: [62.31, 17.38, 13.34],
 94.06: [62.31, 31.75],
 94.

In [9]:
inventory_items_values = get_inventory_items()
        
possible_bet_values = get_possible_bet_values(inventory_items_values)
print(possible_bet_values)

[2025-03-19 12:48:52]  inventory value: 171.78
inventory items gathered: [61.3, 31.75, 20.86, 5.63, 5.51, 3.33, 2.41, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 2.13, 2.0, 1.99, 1.99, 1.99, 1.98, 1.86, 1.33, 1.22, 1.15, 1.12, 1.12, 1.12, 0.87]
[2025-03-19 12:48:52]  there are 29 items in inventory
[2025-03-19 12:48:52]  {0.87: [0.87], 1.12: [1.12], 1.15: [1.15], 1.22: [1.22], 1.33: [1.33], 1.86: [1.86], 1.98: [1.98], 1.99: [1.99], 1.9900000000000002: [1.12, 0.87], 2.0: [2.0], 2.02: [1.15, 0.87], 2.09: [1.22, 0.87], 2.13: [2.13], 2.2: [1.33, 0.87], 2.24: [1.12, 1.12], 2.27: [1.15, 1.12], 2.34: [1.22, 1.12], 2.37: [1.22, 1.15], 2.39: [2.39], 2.41: [2.41], 2.45: [1.33, 1.12], 2.48: [1.33, 1.15], 2.55: [1.33, 1.22], 2.73: [1.86, 0.87], 2.85: [1.98, 0.87], 2.86: [1.99, 0.87], 2.87: [2.0, 0.87], 2.9800000000000004: [1.86, 1.12], 3.0: [2.13, 0.87], 3.01: [1.86, 1.15], 3.08: [1.86, 1.22], 3.1: [1.98, 1.12], 3.1100000000000003: [1.12, 1.12, 0.87], 3.12: [2.0, 1.12], 3.13: [1.98, 1.15], 3.1

In [20]:
bets = []
counter = 0

inventory_items_values = get_inventory_items()
        
possible_bet_values = get_possible_bet_values(inventory_items_values)

while True:

    active_coinflips = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
    coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")
    
    # print(f"{len(coinflip_divs)} active games")
    for coinflip_div in coinflip_row_divs:
        coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
        coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
        
        if "items" in coinflip_items.text:
            # print("more than 5 items...")
            continue
            
        img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
        
        coinflip_items_values = []
        for img in img_elements:
            title = img.get_attribute("data-original-title")
            if "|" in title:
                price_str = title.split("$")[-1]
                price = float(price_str)
                coinflip_items_values.append(price)
    
        possible_tax_value = coinflip_value * 2 * 0.05
        cheapest_item_value = coinflip_items_values[-1]
        
        if cheapest_item_value < possible_tax_value:
            # print(f"not profitable")
            continue
            
        # to join a cf, i need to deposit minimum value
        for bet_value in possible_bet_values:
            if  coinflip_value * 0.9 < bet_value < coinflip_value * 0.92 and coinflip_value not in bets and coinflip_value < 6.0:
                
                ### PRESS JOIN ON SELECTED BET ###
                join_button = coinflip_div.find_element(By.TAG_NAME, "button")
                if "Join" in join_button.text:
                    join_button.click()
                else:
                    continue
                ### PRESS JOIN ON SELECTED BET ###
                
                
                ### SELECT THE ITEMS ###
                inventory_items_div_locator = (By.CLASS_NAME, "depositInventory")
                inventory_items_div = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(inventory_items_div_locator)
                )
                bet_items = copy.deepcopy(possible_bet_values[bet_value])
                try:
                    inventory_items_locator = (By.CLASS_NAME, "InventoryItem")
                    inventory_items = WebDriverWait(driver, 3).until(
                        EC.presence_of_all_elements_located(inventory_items_locator)
                    )
                    
                    # inventory_items_values = []
                    for inventory_item in inventory_items:
                        item_value = float(inventory_item.text.split("$")[1])
                        if item_value in possible_bet_values[bet_value]:
                            inventory_item.click()
                            possible_bet_values[bet_value].remove(item_value)

                except TimeoutException:
                    print(f"Timed out waiting for inventory items")
                    driver.get("https://rustypot.com/coinflip")
                    
                deposit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@onclick='requestCfDeposit()']"))
                )
                deposit_button.click()
                ### SELECT THE ITEMS ###
                
                ### DEPOSIT AND CLOSE STEAM WINDOW ###
                accept_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[@onclick='closeAcceptDeposit()']"))
                )
                accept_button.click()
                
                # Step 2: Wait for the new window/tab to open
                WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
                
                # Step 3: Switch to the new window
                original_window = driver.current_window_handle
                new_window = [w for w in driver.window_handles if w != original_window][0]
                driver.switch_to.window(new_window)
                
                
                try:
                    warning_element = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((
                            By.XPATH,
                            "//*[contains(., 'have reported Blown for attempting trade scams')]"
                        ))
                    )
                    ok_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((
                            By.XPATH,
                            "//div[contains(@class, 'btn_grey_steamui') and span[text()='OK']]"
                            ))
                    )
                    ok_button.click()
                except TimeoutException:
                    print(f"no report appeared")

                
                confirm_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
                )
                confirm_button.click()
                time.sleep(1)
                
                
                confirm_gift_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//div[span[text()='Yes, this is a gift']]"))
                )
                confirm_gift_button.click()
                time.sleep(1)
                
                
                accept_trade_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "trade_confirmbtn"))
                )
                accept_trade_btn.click()
                time.sleep(1)
                
                driver.close()
                driver.switch_to.window(original_window)
                
                ### DEPOSIT AND CLOSE STEAM WINDOW ###
                
                print(f"{coinflip_value=} -> "
                      f"{coinflip_items_values=} -> "
                      f"Bet items= {bet_items}")
                
                bets.append(coinflip_value)
                
                view_button = coinflip_div.find_element(By.XPATH, ".//button[text()='View']")
                view_button.click()
                
                
                # Wait until the <p> inside #fliper-coin disappears
                try:
                    p_element = driver.find_element(By.CSS_SELECTOR, "#fliper-coin > p")
                    WebDriverWait(driver, 120).until(EC.staleness_of(p_element))
                except:
                    # If <p> is already gone, just wait for the new state
                    pass
                
                WebDriverWait(driver, 120).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#fliper-coin .flipper"))
                )

                
                ### wait to check if win or lost ###
                
                # Wait for the parent div to appear
                creator_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "creator-imgs")))
                
                # Wait for the child img.pick within that div
                color_img = WebDriverWait(driver, 10).until(lambda d: creator_div.find_element(By.XPATH, ".//img[@class='pick']"))
                
                # Get the src
                src = color_img.get_attribute("src")
                
                # Extract the color from the filename
                if "Red" in src:
                    creator_color = "Red"
                elif "Black" in src:
                    creator_color = "Black"
                else:
                    creator_color = "Unknown"
                # extract flip color
                
                # 1. Get the 'flipper' element
                flipper = driver.find_element(By.CLASS_NAME, "flipper")
                
                # 2. Get the 'transform' style attribute
                transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"
                
                # 3. Extract the degree
                import re
                match = re.search(r'rotateY\((\d+)deg\)', transform_style)
                if match:
                    degrees = int(match.group(1))
                    # Normalize the rotation within 360°
                    final_angle = degrees % 360
                
                    # Every 180° flips the face, so even number of 180s = same side
                    flips = (degrees // 180) % 2
                
                    flip_face = "Red" if flips == 1 else "Black"
                    print("Coin landed on:", flip_face)
                else:
                    flip_face = "Couldn't determine coin face."
                    print(flip_face)

                # extract flip color
                
                if creator_color == flip_face and (flip_face == "Red" or flip_face == "Black"):
                    print(f"flip was lost!")
                    time.sleep(4) # wait to see the flip

                    driver.get("https://rustypot.com/coinflip")
                    time.sleep(1)
                    print(inventory_items_values)
                    for item in bet_items:
                        inventory_items_values.remove(item)

                    print(f"new inventory items: {inventory_items_values}")
                    possible_bet_values = get_possible_bet_values(inventory_items_values)
                    
                elif flip_face == "Red" or flip_face == "Black":
                    print(f"flip was won!")
                    time.sleep(20) # TODO: implement accept winnings
                    
                    for item in coinflip_items_values:
                        inventory_items_values.append(item)

                    possible_bet_values = get_possible_bet_values(inventory_items_values)
                else:
                    # Couldn't determine coin face.
                    time.sleep(15)
                    
                    driver.get("https://rustypot.com/coinflip")
                    time.sleep(5)
                    inventory_items_values = get_inventory_items()
                    possible_bet_values = get_possible_bet_values(inventory_items_values)
                
                print(f"Back to searching")
    time.sleep(1)

[2025-03-19 21:04:47]  inventory value: 160.85
inventory items gathered: [61.27, 40.35, 31.75, 2.75, 2.39, 2.39, 2.39, 2.39, 2.39, 2.39, 1.99, 1.83, 1.82, 1.7, 1.04, 0.95, 0.53, 0.53]
[2025-03-19 21:04:47]  there are 18 items in inventory
[2025-03-19 21:05:12]  no report appeared
[2025-03-19 21:05:16]  coinflip_value=3.48 -> coinflip_items_values=[3.48] -> Bet items= [1.7, 0.95, 0.53]


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00DE0E93+39155]
	Microsoft::Applications::Events::time_ticks_t::time_ticks_t [0x00C7E3B6+772070]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00A348EE+5182]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A6998A+105610]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A69C7A+106362]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A99B92+302738]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A81EC4+205252]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A63091+78737]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A98530+297008]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A81B86+204422]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A62947+76871]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A61CF5+73717]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A62764+76388]
	sqlite3_dbdata_init [0x00EE65BC+589708]
	Microsoft::Applications::Events:

In [37]:
driver.switch_to.window(original_window)

In [100]:
confirm_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'content') and contains(text(), 'Confirm trade contents')]"))
                )
confirm_button.click()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00DE0E93+39155]
	Microsoft::Applications::Events::time_ticks_t::time_ticks_t [0x00C7E3B6+772070]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00A348EE+5182]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A6998A+105610]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A69C7A+106362]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A99B92+302738]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A81EC4+205252]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A63091+78737]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A98530+297008]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A81B86+204422]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A62947+76871]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A61CF5+73717]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00A62764+76388]
	sqlite3_dbdata_init [0x00EE65BC+589708]
	Microsoft::Applications::Events:

In [38]:
WebDriverWait(driver, 120).until(
    EC.invisibility_of_element_located((By.CSS_SELECTOR, "#fliper-coin > p"))
)

### wait to check if win or lost ###

# wait to flip the coin
time.sleep(3)

# extract our color 
creator_div = driver.find_element(By.CLASS_NAME, "creator-imgs")
color_img = creator_div.find_element(By.XPATH, "//img[@class='pick']")
src = color_img.get_attribute("src")

# Extract the color from the filename
if "Red" in src:
    creator_color = "Red"
elif "Black" in src:
    creator_color = "Black"
else:
    creator_color = "Unknown"
# extract flip color

# 1. Get the 'flipper' element
flipper = driver.find_element(By.CLASS_NAME, "flipper")

# 2. Get the 'transform' style attribute
transform_style = flipper.get_attribute("style")  # e.g. "transform: rotateY(1620deg);"

# 3. Extract the degree
import re
match = re.search(r'rotateY\((\d+)deg\)', transform_style)
if match:
    degrees = int(match.group(1))
    # Normalize the rotation within 360°
    final_angle = degrees % 360

    # Every 180° flips the face, so even number of 180s = same side
    flips = (degrees // 180) % 2

    flip_face = "Red" if flips == 1 else "Black"
    print("Coin landed on:", flip_face)
else:
    flip_face = "Couldn't determine coin face."
    print(flip_face)

# extract flip color

if creator_color == flip_face:
    print(f"flip was lost!")
else:
    print(f"flip was won!")

[2025-03-18 18:42:46]  Coin landed on: Black
[2025-03-18 18:42:46]  flip was won!
